# 07에서 추가 학습하기

# 07
## 06에서 모델을 11x로 변경해서 돌리기
## batch=64 (주피터 창 강제 종료됨)
## => batch=32
- OutOfMemoryError: CUDA out of memory. Tried to allocate 76.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 22.19 GiB is allocated by PyTorch, and 136.10 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.
## => batch=16 (소요시간 너무 긺)
## => batch=8

In [1]:
# 📌 [기본 패키지 임포트]
import os
import shutil
import torch
import ultralytics
from ultralytics import YOLO
import time
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
from glob import glob
from collections import Counter
import threading

In [2]:
# 📌 [1] GPU 및 환경 체크
print("✅ CUDA 사용 가능:", torch.cuda.is_available())
print("✅ PyTorch 버전:", torch.__version__)
ultralytics.checks()

Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
Setup complete  (24 CPUs, 31.7 GB RAM, 25.5/735.1 GB disk)


In [3]:
# 📌 [2] 실패한 이전 runs 삭제
def clean_incomplete_runs(base_dir='../code/runs/detect'):
    if not os.path.exists(base_dir):
        print(f"📂 '{base_dir}' 폴더가 없어 삭제 생략")
        return

    deleted = False
    for folder in os.listdir(base_dir):
        path = os.path.join(base_dir, folder)
        best_model = os.path.join(path, 'weights', 'best.pt')
        if not os.path.exists(best_model):
            print(f"🧹 삭제 대상: {path}")
            shutil.rmtree(path)
            deleted = True

    if not deleted:
        print("✅ 삭제할 미완성 run 없음. 모든 run이 best.pt 포함.")

clean_incomplete_runs()

✅ 삭제할 미완성 run 없음. 모든 run이 best.pt 포함.


In [4]:
# 📌 [3] 가장 최근 run 결과 요약 출력
def get_last_run_csv():
    runs = sorted(glob('runs/detect/*/results.csv'), key=os.path.getmtime, reverse=True)
    if runs:
        print(f"📄 마지막 훈련 결과 파일: {runs[0]}")
        df = pd.read_csv(runs[0])
        print("🔁 마지막 훈련 결과 (마지막 5줄):")
        print(df.tail())
    else:
        print("ℹ️ 이전 훈련 결과 없음.")

get_last_run_csv()

📄 마지막 훈련 결과 파일: runs/detect\yolov11x_custom_001\results.csv
🔁 마지막 훈련 결과 (마지막 5줄):
    epoch     time  train/box_loss  train/cls_loss  train/dfl_loss  \
25     26  22523.5         0.53828         0.44147         0.95278   
26     27  23074.8         0.53529         0.41576         0.95056   
27     28  23978.3         0.52362         0.39257         0.94227   
28     29  25021.9         0.51856         0.37470         0.93696   
29     30  26101.2         0.50880         0.36424         0.93301   

    metrics/precision(B)  metrics/recall(B)  metrics/mAP50(B)  \
25               0.94191            0.87008           0.94760   
26               0.93775            0.87999           0.94480   
27               0.91825            0.89548           0.95111   
28               0.96195            0.89026           0.95894   
29               0.93677            0.90895           0.95819   

    metrics/mAP50-95(B)  val/box_loss  val/cls_loss  val/dfl_loss    lr/pg0  \
25              0.81856    

In [5]:
# 📌 [4] 자동 넘버링 run 이름 생성
def get_next_run_name(base_dir='runs/detect', prefix='yolov11x_custom'):
    i = 1
    while os.path.exists(os.path.join(base_dir, f'{prefix}_{i:03d}')):
        i += 1
    return f'{prefix}_{i:03d}'

run_name = get_next_run_name()
print(f"🚀 새 훈련 run 이름: {run_name}")

🚀 새 훈련 run 이름: yolov11x_custom_002


In [6]:
# 📌 [5] 입력 데이터 정보 출력 및 클래스별 레이블 개수 확인 함수 추가
def get_dataset_info(img_dir, label_dir, class_names=None):
    img_files = glob(os.path.join(img_dir, '*.jpg'))
    label_files = glob(os.path.join(label_dir, '*.txt'))

    total_img_size = sum(os.path.getsize(f) for f in img_files)
    total_label_size = sum(os.path.getsize(f) for f in label_files)

    print(f"\n📁 이미지 경로: {img_dir}")
    print(f"🖼️ 이미지 개수: {len(img_files)}")
    print(f"💾 이미지 용량: {total_img_size / 1024 / 1024:.2f} MB")
    print(f"📄 라벨 경로: {label_dir}")
    print(f"📝 라벨 개수: {len(label_files)}")
    print(f"💾 라벨 용량: {total_label_size / 1024:.2f} KB")

    # 클래스별 라벨 개수 집계
    classes = []
    for f in label_files:
        with open(f, 'r') as file:
            lines = file.readlines()
            classes += [int(line.split()[0]) for line in lines if line.strip()]
    counter = Counter(classes)
    print(f"\n📊 클래스별 라벨 개수 ({label_dir}):")
    if class_names:
        for cls_idx, count in sorted(counter.items()):
            print(f"  {cls_idx} ({class_names[cls_idx]}): {count}")
    else:
        print(counter)

# 예시로 data.yaml에서 클래스명 읽어오기
import yaml
with open('data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)
class_names = data_cfg.get('names', None)

get_dataset_info('../train/images', '../train/labels', class_names)
get_dataset_info('../valid/images', '../valid/labels', class_names)


📁 이미지 경로: ../train/images
🖼️ 이미지 개수: 3530
💾 이미지 용량: 67.55 MB
📄 라벨 경로: ../train/labels
📝 라벨 개수: 3530
💾 라벨 용량: 319.50 KB

📊 클래스별 라벨 개수 (../train/labels):
  0 (Green Light): 542
  1 (Red Light): 585
  2 (Speed Limit 10): 19
  3 (Speed Limit 100): 267
  4 (Speed Limit 110): 101
  5 (Speed Limit 120): 252
  6 (Speed Limit 20): 285
  7 (Speed Limit 30): 334
  8 (Speed Limit 40): 235
  9 (Speed Limit 50): 283
  10 (Speed Limit 60): 301
  11 (Speed Limit 70): 318
  12 (Speed Limit 80): 323
  13 (Speed Limit 90): 168
  14 (Stop): 285

📁 이미지 경로: ../valid/images
🖼️ 이미지 개수: 801
💾 이미지 용량: 15.38 MB
📄 라벨 경로: ../valid/labels
📝 라벨 개수: 801
💾 라벨 용량: 69.86 KB

📊 클래스별 라벨 개수 (../valid/labels):
  0 (Green Light): 122
  1 (Red Light): 108
  3 (Speed Limit 100): 52
  4 (Speed Limit 110): 17
  5 (Speed Limit 120): 60
  6 (Speed Limit 20): 56
  7 (Speed Limit 30): 74
  8 (Speed Limit 40): 55
  9 (Speed Limit 50): 71
  10 (Speed Limit 60): 76
  11 (Speed Limit 70): 78
  12 (Speed Limit 80): 56
  13 (Speed Limit 

In [ ]:
# 📌 캐시 파일 삭제 함수
# def safe_delete_cache(path: str):
#     if not os.path.exists(path):
#         print(f"🟢 캐시 파일 없음 (삭제 불필요): {path}")
#         return

#     file_size = os.path.getsize(path)
#     last_modified = os.path.getmtime(path)
#     now = time.time()

#     should_delete = (
#         file_size < 1024 or  # 1KB 이하
#         (now - last_modified > 3600)  # 1시간 이상 경과
#     )

#     if should_delete:
#         try:
#             os.remove(path)
#             print(f"✅ 캐시 삭제 완료: {path}")
#         except Exception as e:
#             print(f"❌ 캐시 삭제 실패: {path} → {e}")
#     else:
#         print(f"🔒 캐시 유지: {path} (조건 미충족)")

# safe_delete_cache(r"D:\Project\PJT_07\train\labels.cache")
# safe_delete_cache(r"D:\Project\PJT_07\valid\labels.cache")

In [7]:
print(run_name)

yolov11x_custom_002


In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# GPU 메모리 사용량 주기적으로 출력 (간격 60초로 조정)
def gpu_monitor():
    while True:
        if torch.cuda.is_available():
            mem_alloc = torch.cuda.memory_allocated() / 1024**2
            mem_reserved = torch.cuda.memory_reserved() / 1024**2
            print(f"[GPU 모니터] 메모리 할당: {mem_alloc:.1f}MB, 예약: {mem_reserved:.1f}MB")
        time.sleep(60)  # 60초 간격

monitor_thread = threading.Thread(target=gpu_monitor, daemon=True)
monitor_thread.start()

# 모델 로드
#model = YOLO('yolo11x.pt')
model = YOLO('runs/detect/yolov11x_custom_001/weights/best.pt')  # 기존 모델에서 이어서 학습 시

# 하이퍼파라미터 튜닝 포함한 train 함수 호출
start_time = time.time()

model.train(
    data='data.yaml',
    epochs=30,
    imgsz=640,
    name=run_name,
    device='0',
    workers=0,
    verbose=True,  # 더 자세한 로그 출력

    batch=8,
    patience=10,
    warmup_epochs=0  # 웜업 비활성화
)

train_duration = time.time() - start_time
print(f"✅ 훈련 소요 시간: {train_duration:.2f}초")

[GPU 모니터] 메모리 할당: 0.0MB, 예약: 0.0MB
New https://pypi.org/project/ultralytics/8.3.157 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/yolov11x_custom_001/weights/best.pt, momentum=0.937, mosaic=1.0, mult

train: Scanning D:\Project\PJT_07\train\labels.cache... 3530 images, 3 backgrounds, 0 corrupt: 100%|██████████| 3530

val: Fast image access  (ping: 0.10.0 ms, read: 25.710.9 MB/s, size: 20.7 KB)



val: Scanning D:\Project\PJT_07\valid\labels.cache... 801 images, 0 backgrounds, 0 corrupt: 100%|██████████| 801/801

Plotting labels to runs\detect\yolov11x_custom_002\labels.jpg... 


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00

                   all        801        944      0.946      0.939      0.972      0.847



30 epochs completed in 9.684 hours.
Optimizer stripped from runs\detect\yolov11x_custom_002\weights\last.pt, 114.4MB
Optimizer stripped from runs\detect\yolov11x_custom_002\weights\best.pt, 114.4MB

Validating runs\detect\yolov11x_custom_002\weights\best.pt...
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
YOLO11x summary (fused): 190 layers, 56,844,349 parameters, 0 gradients, 194.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00


                   all        801        944      0.946      0.939      0.972      0.847
           Green Light         87        122      0.863      0.777      0.892       0.55
             Red Light         74        108      0.865      0.796      0.855      0.545
       Speed Limit 100         52         52       0.98      0.981      0.992      0.918
       Speed Limit 110         17         17      0.891      0.941       0.99      0.906
       Speed Limit 120         60         60      0.984      0.983      0.994      0.924
        Speed Limit 20         56         56      0.948      0.982      0.987       0.87
        Speed Limit 30         71         74      0.918      0.973      0.991      0.935
        Speed Limit 40         53         55      0.986      0.945      0.989      0.883
        Speed Limit 50         68         71      0.971       0.95      0.988      0.884
        Speed Limit 60         76         76      0.961      0.962      0.986      0.887
        Speed Limit 7

In [ ]:
# 📌 [7] 검증 수행
# start_val = time.time()

# model = YOLO(f'runs/detect/{run_name}/weights/best.pt')
# results = model.val()
# val_duration = time.time() - start_val
# print(f"✅ 검증 소요 시간: {val_duration:.2f}초")
# print(f"mAP50: {results.box.map50:.4f}, mAP50-95: {results.box.map:.4f}")

# # PR Curve 시각화 (val 결과 내 PR curve 있으면)
# if hasattr(results, 'plots') and 'pr_curve' in results.plots:
#     plt.figure(figsize=(8,6))
#     plt.imshow(results.plots['pr_curve'])
#     plt.axis('off')
#     plt.title("PR Curve")
#     plt.show()

In [9]:
# 'yolov11x_custom_001'
print(run_name)

yolov11x_custom_002


In [10]:
# 📌 [8] 예측 실행 및 저장
test_img = '../test/images/00014_00004_00020_png.rf.e3b35f3ca153fde99af4db4a67d2a564.jpg'
pred = model.predict(
    source=test_img,
    conf=0.25,
    save=True,
    project='runs/predict/custom_test',
    name=run_name,
    exist_ok=True
)


image 1/1 D:\Project\PJT_07\code\..\test\images\00014_00004_00020_png.rf.e3b35f3ca153fde99af4db4a67d2a564.jpg: 640x640 1 Stop, 22.7ms
Speed: 2.4ms preprocess, 22.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\predict\custom_test\yolov11x_custom_002


In [11]:
# 📌 [9] 예측 결과 시각화 및 GT bbox와 비교 함수 추가
def draw_boxes_with_gt(image_path, pred_results, gt_label_path, class_names):
    img = cv2.imread(image_path)
    ih, iw = img.shape[:2]

    # GT 박스 (파란색)
    with open(gt_label_path, 'r') as f:
        gt_lines = f.readlines()

    for line in gt_lines:
        cls, x, y, w, h = map(float, line.strip().split())
        x1 = int((x - w/2) * iw)
        y1 = int((y - h/2) * ih)
        x2 = int((x + w/2) * iw)
        y2 = int((y + h/2) * ih)
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,0,0), 2)
        cv2.putText(img, f"GT: {class_names[int(cls)]}", (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)

    # 예측 박스 (초록색)
    for r in pred_results:
        boxes = r.boxes.xyxy.cpu().numpy().astype(int)
        scores = r.boxes.conf.cpu().numpy()
        classes = r.boxes.cls.cpu().numpy().astype(int)
        for (x1,y1,x2,y2), conf, cls in zip(boxes, scores, classes):
            label = f"{class_names[cls]} {conf:.2f}"
            cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
            (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            cv2.rectangle(img, (x1, y1 - th - 5), (x1 + tw, y1), (0,255,0), -1)
            cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

    plt.figure(figsize=(12,12))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("예측 박스 (초록) 와 GT 박스 (파랑) 비교")
    plt.show()

gt_label_for_test_img = '../test/labels/00014_00004_00020_png.rf.e3b35f3ca153fde99af4db4a67d2a564.txt'
draw_boxes_with_gt(test_img, pred, gt_label_for_test_img, class_names)

<Figure size 1200x1200 with 1 Axes>

In [12]:
# 📌 [10] ONNX 모델로 내보내기
export_path = model.export(format='onnx')
print(f"🧠 ONNX 모델 저장 경로: {export_path}")

Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CPU (13th Gen Intel Core(TM) i7-13700HX)

PyTorch: starting from 'runs\detect\yolov11x_custom_002\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 19, 8400) (109.1 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.57...
ONNX: export success  7.3s, saved as 'runs\detect\yolov11x_custom_002\weights\best.onnx' (217.2 MB)

Export complete (9.0s)
Results saved to D:\Project\PJT_07\code\runs\detect\yolov11x_custom_002\weights
Predict:         yolo predict task=detect model=runs\detect\yolov11x_custom_002\weights\best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs\detect\yolov11x_custom_002\weights\best.onnx imgsz=640 data=data.yaml  
Visualize:       https://netron.app
🧠 ONNX 모델 저장 경로: runs\detect\yolov11x_custom_002\weights\best.onnx


In [13]:
# 📌 [11] ONNX 모델 기반 예측 시각화 (기존 코드 유지)
onnx_model = YOLO(os.path.join('runs/detect', run_name, 'weights', 'best.onnx'))

image = cv2.imread(test_img)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
resized_image = cv2.resize(image, (640, 640))

results_onnx = onnx_model(resized_image, imgsz=640)

for result in results_onnx:
    boxes = result.boxes
    names = result.names
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0]
        cls = int(box.cls[0])
        label = f"{names[cls]} {conf:.2f}"

        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(image, (x1, y1 - text_height - 5), (x1 + text_width, y1), (0, 255, 0), -1)
        cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

plt.figure(figsize=(12, 12))
plt.imshow(image)
plt.axis('off')
plt.title("🔍 ONNX 예측 결과")
plt.show()

Loading runs\detect\yolov11x_custom_002\weights\best.onnx for ONNX Runtime inference...
Using ONNX Runtime CUDAExecutionProvider

0: 640x640 1 Stop, 170.4ms
Speed: 2.4ms preprocess, 170.4ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 1200x1200 with 1 Axes>

In [ ]:
# print(df.columns)

In [14]:
# 📌 [12] 훈련 결과 시각화 (mAP/Loss 등)
results_csv = f'runs/detect/{run_name}/results.csv'

if os.path.exists(results_csv):
    df = pd.read_csv(results_csv)
    print(f"📈 훈련 결과 요약 (Epoch별 mAP):\n{df[['metrics/mAP50(B)', 'metrics/mAP50-95(B)']].tail()}")

    # Box Loss 그래프
    df[['train/box_loss', 'val/box_loss']].plot(title='Box Loss', figsize=(6, 4))
    plt.savefig(f'runs/detect/{run_name}/box_loss.png')
    plt.show()

    # ✅ 수정: train/valid 전체 손실 시각화 (box, cls, dfl)
    df[['train/box_loss', 'train/cls_loss', 'train/dfl_loss',
        'val/box_loss', 'val/cls_loss', 'val/dfl_loss']].plot(figsize=(10,6))
    plt.title('Train & Valid Losses (Box, Class, DFL)')
    plt.savefig(f'runs/detect/{run_name}/losses.png')
    plt.show()

    # mAP, Precision, Recall 시각화
    df[['metrics/mAP50(B)', 'metrics/precision(B)', 'metrics/recall(B)']].plot(figsize=(10,6))
    plt.title('Metrics: mAP50, Precision, Recall')
    plt.savefig(f'runs/detect/{run_name}/metrics.png')
    plt.show()

    best_idx = df['metrics/mAP50-95(B)'].idxmax()
    print("🎯 최고 성능 Epoch:")
    print(df.loc[best_idx, ['epoch', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']])
else:
    print("📛 results.csv 없음 (훈련 실패?)")

📈 훈련 결과 요약 (Epoch별 mAP):
    metrics/mAP50(B)  metrics/mAP50-95(B)
25           0.96780              0.83444
26           0.97017              0.83821
27           0.97174              0.83894
28           0.97370              0.84421
29           0.97186              0.84689


<Figure size 600x400 with 1 Axes>

<Figure size 1000x600 with 1 Axes>

<Figure size 1000x600 with 1 Axes>

🎯 최고 성능 Epoch:
epoch                  30.00000
metrics/mAP50(B)        0.97186
metrics/mAP50-95(B)     0.84689
Name: 29, dtype: float64


In [15]:
# 📌 [13] 결과를 JSON으로 저장
def save_results_to_json(csv_path, json_path):
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        result_data = df.to_dict(orient='records')
        with open(json_path, 'w') as f:
            json.dump(result_data, f, indent=4)
        print(f"📦 결과 JSON 저장 완료: {json_path}")

save_results_to_json(
    csv_path=f'runs/detect/{run_name}/results.csv',
    json_path=f'runs/detect/{run_name}/results_summary.json'
)

📦 결과 JSON 저장 완료: runs/detect/yolov11x_custom_002/results_summary.json
